multiprocessing本身是Python的多进程库，用来处理与多进程相关的操作。但是由于进程与进程之间不能直接共享内存和堆栈资源，而且启动新的进程开销也比线程大得多，因此使用多线程来爬取比使用多进程有更多的优势。multiprocessing下面有一个dummy模块，它可以让Python的线程使用multiprocessing的各种方法。

dummy下面有一个Pool类，它用来实现线程池。这个线程池有一个map()方法，可以让线程池里面的所有线程都“同时”执行一个函数。

### 1. 简单示例

In [1]:
from multiprocessing.dummy import Pool

# 计算平方
def calc_power2(num):
    return num * num
# 初始化3个线程的线程池
pool = Pool(3)
origin_num = [x for x in range(10)]
result = pool.map(calc_power2, origin_num)
print(f'计算0-9的平方分别为：{result}')

计算0-9的平方分别为：[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


### 2. 开放多线程爬虫

爬虫是I/O密集型的操作，特别是在请求网页源代码的时候，如果使用单线程来开发，会浪费大量的时间来等待网页返回，所以把多线程技术应用到爬虫中，可以大大提高爬虫的运行效率。

#### （1）使用单线程循环访问百度首页100，计算总时间

In [2]:
import requests
import time

def query(url):
    requests.get(url)
url = 'https://baidu.com'
start = time.time()
for i in range(100):
    query(url)
end = time.time()
print(f'单线程循环访问100次百度首页，耗时：{end - start}')

单线程循环访问100次百度首页，耗时：56.42745542526245


#### (2) 使用5个线程访问100次百度首页，计算总时间

In [3]:
start = time.time()
url_list = []
for i in range(100):
    url_list.append(url)
pool = Pool(5)
pool.map(query, url_list)
end = time.time()
print(f'5线程循环访问100次百度首页，耗时：{end - start}')

5线程循环访问100次百度首页，耗时：13.107457399368286


#### 线程池并不是设置得越大越好。从上面的结果也可以看到，5个线程运行的时间其实比一个线程运行时间的五分之一要多一点。这多出来的一点其实就是线程切换的时间。这也从侧面反映了Python的多线程在微观上还是串行的。因此，如果线程池设置得过大，线程切换导致的开销可能会抵消多线程带来的性能提升。线程池的大小需要根据实际情况来确定，并没有确切的数据。